In [1]:
!pip3 install spacy -q
!python3 -m spacy download en -q
!python3 -m spacy download en_core_news_md -q

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')

✘ No compatible package found for 'en_core_news_md' (spaCy v3.2.4)



### importer les differentes librairies

In [2]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from urllib.parse import urlparse

### lire les données et transformer les colonnes majuscules en minuscules

In [3]:
df = pd.read_csv('/home/jovyan/Projects/projet_perso/train.csv')
df.columns = df.columns.str.lower()
df.sample(10)

,labels,text,text_tag
4806,2,Wisconsin does not have a deficit. Thanks to R...,"deficit,state-budget,state-finances,taxes"
2195,5,"In 2009, Democratic Wisconsin state senators r...","labor,state-budget,taxes"
9901,5,Georgia has recovered more than $60 million th...,medicaid
3321,2,Ive never supported legislation that invades p...,"abortion,sexuality"
7817,3,"Since (welfare reform) was signed into law, th...",welfare
3734,2,Live cats have holes drilled into their skulls...,"animals,education,health-care,public-health,sc..."
3280,2,More than 80 percent of Georgians dont support...,guns
4348,0,"In Texas public schools, we spend an average o...","education,state-budget,state-finances,states"
1119,1,Says Ron Johnson has essentially done nothing ...,"candidates-biography,drugs,homeland-security,i..."
5903,3,Proposed cuts in the House farm bill mean 2 mi...,"agriculture,hunger,welfare"


### Filtrer les données par rapport aux labels qui sont différents de 4 (4 = Iconnu)

In [4]:
df = df[df["labels"] != 4]

### supprimer les valeurs manquantes

In [5]:
df.isnull().any()
df["text_tag"].isnull().sum()
df=df.dropna()
df.shape

(9399, 3)

### transformer les labels multivariés en bivariés ( 1 = Fake news, 0 = vrai info)

In [6]:
df["labels"] =df["labels"].apply(lambda x : 0 if x == 3 or x == 5
                       else 1)
df["labels"].value_counts()

1    5761
0    3638
Name: labels, dtype: int64

In [7]:
df.sample(10)

,labels,text,text_tag
4543,0,"Over the past 35 years, Florida families have ...","environment,housing,states,weather"
9755,0,Obama promised to cut the deficit by half by t...,"deficit,message-machine-2012"
1279,1,Unemployment among Oregon high school graduate...,"economy,jobs,workers"
7057,0,"30,000 Arizonans may have no choice on Afforda...",health-care
9687,0,Then-Sen. Barack Obama refused to raise the de...,"candidates-biography,federal-budget"
9302,0,Says Connie Mack was the only member of the Fl...,"space,voting-record"
7169,0,"The Buffett rule raises virtually no money, ma...","deficit,economy,federal-budget,taxes"
1236,1,Gov. Chris Christie owes the state money for a...,state-budget
8537,0,"For every $1 spent on mitigation, $4 in post-s...",government-efficiency
8596,0,@Jorge_Elorza still think your plan is safe? R...,"public-safety,unions"


### transformer tous les tag_text majuscules en minuscules

In [8]:
df["text_tag"] = df["text_tag"].fillna("").str.lower()

In [9]:
tags = df.text_tag.unique()
tags

array(['abortion', 'energy,history,job-accomplishments', 'foreign-policy',
       ..., 'animals,elections', 'retirement,social-security',
       'florida,foreign-policy'], dtype=object)

### rajouter les tag_text au text

In [10]:
df

,labels,text,text_tag
0,1,Says the Annies List political group supports ...,abortion
1,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,1,The economic turnaround started at the end of ...,"economy,jobs"
...,...,...,...
10234,1,"Under the ruling of the Supreme Court, any lob...","corporations,elections"
10235,0,There are a larger number of shark attacks in ...,"animals,elections"
10236,0,Democrats have now become the party of the [At...,elections
10237,1,Says an alternative to Social Security that op...,"retirement,social-security"


In [11]:
df["text"] = df["text"] + " " + df["text_tag"].str.replace(",", " ")

In [12]:
df["text"][110]

'Two-thirds of Wisconsinites receiving unemployment checks are not required to search for work due to current work search exemptions. government-regulation jobs poverty welfare'

supprimer la colonnes text_tag

In [13]:
df = df.drop(columns = "text_tag")
df.head()

,labels,text
0,1,Says the Annies List political group supports ...
1,1,When did the decline of coal start? It started...
2,0,"Hillary Clinton agrees with John McCain ""by vo..."
3,1,Health care reform legislation is likely to ma...
4,1,The economic turnaround started at the end of ...


In [14]:
df["text"][1]

'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration. energy history job-accomplishments'

### preprocessing à l'aide de spacy avec les spécificités de la langues anglais

In [15]:
import en_core_web_sm
nlp = en_core_web_sm.load()

### transformer notre colonne text en un objet nlp

In [16]:
df["text_tokens"] = df.text.map(lambda x: nlp(x))
df["text_tokens"]

0        (Says, the, Annies, List, political, group, su...
1        (When, did, the, decline, of, coal, start, ?, ...
2        (Hillary, Clinton, agrees, with, John, McCain,...
3        (Health, care, reform, legislation, is, likely...
4        (The, economic, turnaround, started, at, the, ...
                               ...                        
10234    (Under, the, ruling, of, the, Supreme, Court, ...
10235    (There, are, a, larger, number, of, shark, att...
10236    (Democrats, have, now, become, the, party, of,...
10237    (Says, an, alternative, to, Social, Security, ...
10238    (On, lifting, the, U.S., Cuban, embargo, and, ...
Name: text_tokens, Length: 9399, dtype: object

### transformations en lemmes et en pos via spacy

In [17]:
tokens = df.text_tokens.tolist()
tokens = [(i.lemma_, i.pos_, i.text) for s in tokens for i in s]
tokens = pd.DataFrame(tokens, columns = ["lemma", "pos", "text"])
tokens.head()

,lemma,pos,text
0,say,VERB,Says
1,the,DET,the
2,Annies,PROPN,Annies
3,List,PROPN,List
4,political,ADJ,political


In [18]:
tokens.shape

(232113, 3)

In [19]:
for tok in df.text_tokens.iloc[0]:
    print(tok.lemma_, tok.pos_, tok.text)

say VERB Says
the DET the
Annies PROPN Annies
List PROPN List
political ADJ political
group NOUN group
support VERB supports
third ADJ third
- PUNCT -
trimester NOUN trimester
abortion NOUN abortions
on ADP on
demand NOUN demand
. PUNCT .
abortion NOUN abortion


In [20]:
tokens.pos.unique()

array(['VERB', 'DET', 'PROPN', 'ADJ', 'NOUN', 'PUNCT', 'ADP', 'SCONJ',
       'PRON', 'PART', 'AUX', 'NUM', 'ADV', 'SYM', 'CCONJ', 'INTJ',
       'SPACE', 'X'], dtype=object)

### filtrer par pos
#### ici nous avons choisir de filtrer par les pos suivants verb/nom/adj/adv/pron/aux

In [21]:
def lemmas_filtered_on_pos(tokens, pos_list=["VERB", "NOUN", "ADJ", "ADV",'PRON','AUX','PROPN','ADP']):
    return " ".join([t.lemma_ for t in tokens if t.pos_ in pos_list])

In [22]:
df["text_cleaned"] = df.text_tokens.map(lemmas_filtered_on_pos)

In [23]:
df["text_cleaned"][0]

'say Annies List political group support third trimester abortion on demand abortion'

In [24]:
len(df["text_cleaned"])

9399

In [25]:
df.head()

,labels,text,text_tokens,text_cleaned
0,1,Says the Annies List political group supports ...,"(Says, the, Annies, List, political, group, su...",say Annies List political group support third ...
1,1,When did the decline of coal start? It started...,"(When, did, the, decline, of, coal, start, ?, ...",do decline of coal start it start natural gas ...
2,0,"Hillary Clinton agrees with John McCain ""by vo...","(Hillary, Clinton, agrees, with, John, McCain,...",Hillary Clinton agree with John McCain by vote...
3,1,Health care reform legislation is likely to ma...,"(Health, care, reform, legislation, is, likely...",health care reform legislation be likely manda...
4,1,The economic turnaround started at the end of ...,"(The, economic, turnaround, started, at, the, ...",economic turnaround start at end of my term ec...


In [28]:
all_text = " ".join(df["text_cleaned"].tolist())
len(set(all_text.split(" ")))

9605

In [27]:
import pickle
df.to_pickle('df_lem_ftr1.pk')